In [1]:
from tqdm import tqdm
import csv
import requests
import json
import sqlite3
from datetime import datetime

In [2]:
response = requests.get('https://api.fbi.gov/wanted/v1/list')
data = json.loads(response.content)    
data['items'][2]

{'scars_and_marks': None,
 'possible_states': None,
 'complexion': None,
 'files': [{'name': 'English',
   'url': 'https://www.fbi.gov/wanted/murders/omar-alexander-cardenas/download.pdf'}],
 'field_offices': ['losangeles'],
 'race_raw': 'White (Hispanic)',
 'legat_names': None,
 'modified': '2021-12-09T16:50:52+00:00',
 'remarks': '<p>Cardenas often has a beard and wears prescription glasses.</p>',
 'weight': 'Approximately 240 to 300 pounds',
 'publication': '2021-10-29T13:24:00',
 'status': 'na',
 'race': 'hispanic',
 'suspects': None,
 'hair': 'brown',
 'age_min': None,
 'occupations': None,
 'images': [{'thumb': 'https://www.fbi.gov/wanted/murders/omar-alexander-cardenas/@@images/image/thumb',
   'original': 'https://www.fbi.gov/wanted/murders/omar-alexander-cardenas/@@images/image',
   'large': 'https://www.fbi.gov/wanted/murders/omar-alexander-cardenas/@@images/image/large',
   'caption': 'Photograph taken in 2018'},
  {'thumb': 'https://www.fbi.gov/wanted/murders/omar-alexander

In [3]:
# for i in range(len(data['items'])):
#     print(data['items'])

In [8]:
import requests
import json
from datetime import datetime

d_list = []

for j in range(55):
    response = requests.get('https://api.fbi.gov/wanted/v1/list', params={
    'page': j
    })
    data = json.loads(response.content)    
    for i in range(len(data['items'])):        
        if (data['items'][i]['subjects'] != []) and ('murder' not in data['items'][i]['url']) and ('destructive' not in data['items'][i]['url']):
            d = {}
            
            date_format = "%Y-%m-%d"

            pub_date = data['items'][i]['publication'].split('T', 1)[0]
            pub_date = datetime.strptime(pub_date,date_format)
            
            death_list = ['died', 'homicide', 'dead', 'death', 'murder', 'murdered', 'remains were recovered', 'killed', 'body was recovered', 'bomb', 'killing', 'remains were located']

            if ((data['items'][i]['subjects'][0] == 'Seeking Information') and (data['items'][i]['details'] is not None) and ('missing' in data['items'][i]['details'].lower())) or (data['items'][i]['subjects'][0] == 'Kidnappings and Missing Persons') or (data['items'][i]['subjects'][0] == 'ViCAP Missing Persons'):
                if (pub_date <= datetime.strptime("2021-10-01", date_format)) and (data['items'][i]['details'] is not None) and (any(word.lower() in data['items'][i]['details'].lower() for word in death_list) == False):
                    d['subject'] = data['items'][i]['subjects'][0]
                    d['name'] = data['items'][i]['title'].title()
                    
                    d['date_of_birth'] = data['items'][i]['dates_of_birth_used']
                    
                    d['place_of_birth'] = data['items'][i]['place_of_birth']
                
                    if data['items'][i]['age_range'] is not None:
                        d['age'] = data['items'][i]['age_range']
                    elif data['items'][i]['age_range'] is None:
                        d['age'] = data['items'][i]['age_max']
                    
                    d['sex'] = data['items'][i]['sex']
                    
                    d['height'] = data['items'][i]['height_min']

                    if data['items'][i]['race_raw'] is not None:
                        d['race'] = data['items'][i]['race_raw'].title()
                    else:
                        d['race'] = data['items'][i]['race']
                        
                    if data['items'][i]['hair_raw'] is not None:
                        d['hair'] = data['items'][i]['hair_raw']
                    else:
                        d['hair'] = data['items'][i]['hair']
                        
                    if data['items'][i]['eyes'] is not None:
                        d['eyeColor'] = data['items'][i]['eyes'].title()
                    else:
                        d['eyeColor'] = data['items'][i]['eyes']
                        
                    d['scars_and_marks'] = data['items'][i]['scars_and_marks']
                    
                    d['publication'] = data['items'][i]['publication'].split('T', 1)[0]
                    
                    d['details'] = data['items'][i]['details'].replace('<p>', '').replace('</p>', '')
                    
                    d['description'] = data['items'][i]['description']
                        
                    if (data['items'][i]['field_offices'] != []):
                        d['fieldOffices'] = data['items'][i]['field_offices']
                        
                    d['url'] = data['items'][i]['url']
                    d['image'] = data['items'][i]['images'][0]['original']
                    d['page'] = data['page']
                    d_list.append(d)
            

In [9]:
d_list

[{'subject': 'Kidnappings and Missing Persons',
  'name': 'Katelin Michelle Akens',
  'date_of_birth': None,
  'place_of_birth': None,
  'age': '19 years old (at time of disappearance)',
  'sex': 'Female',
  'height': 64,
  'race': 'White',
  'hair': 'Blonde',
  'eyeColor': 'Blue',
  'scars_and_marks': 'Akens has a tattoo of five butterflies on her arm and a tattoo of three stars on her right foot. She has gauge earrings in her earlobes, a navel piercing, a nose piercing, and a lip piercing.',
  'publication': '2018-02-26',
  'details': 'Katelin Michelle Akens was last seen on December 5, 2015, in the vicinity of Oak Crest Drive in Partlow, Virginia.\xa0 She was visiting family in Spotsylvania County, Virginia, and was scheduled to fly home to Arizona on December 5, 2015.\xa0 However, she did not board her flight and has not been seen since.\xa0 On December 7, 2015, the Virginia Department of Transportation (VDOT) recovered a piece of blue luggage from a roadside ditch, unzipped, empty

In [10]:
len(d_list)

167

In [7]:
def write_csv(filename, data):
    '''
    Write <data> to the .csv file specified by <filename>

    Parameters
    ----------
    filename: string
        the name of the csv file which will store the data
    data: list
        list of dictionaries to convert to csv

    Returns
    -------
    None
    '''

    keys = data[0].keys()
    a_file = open(filename, "w")
    dict_writer = csv.DictWriter(a_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(data)
    a_file.close()


In [8]:
### BUILD DATABASE & TABLES ###

def create_db():
    '''
    Create database to hold information for movies and directors with two tables: movieInfo and director.

    Parameters
    ----------
    None

    Returns
    -------
    None
    '''

    conn = sqlite3.connect('fbi.db')
    cur = conn.cursor()

    # drop tables first to prevent issues during testing
    drop_table = '''
        DROP TABLE IF EXISTS missingPerson;
    '''
    
    missing_person_table = '''
        CREATE TABLE missingPerson (
            id INTEGER PRIMARY KEY AUTOINCREMENT, 
            subject VARCHAR(255),
            name VARCHAR(255),
            dateOfBirth VARCHAR(255),
            placeOfBirth VARCHAR(255),
            age VARCHAR(255),
            sex VARCHAR(255),
            height VARCHAR(255),
            race VARCHAR(255),
            hair VARCHAR(255),
            eyeColor VARCHAR(255),
            scarsMarks VARCHAR(255),
            publication DATE,
            details VARCHAR(255),
            description VARCHAR(255)
            fieldOffices VARCHAR(255),
            url VARCHAR(255),
            image VARCHAR(255),
            page INTEGER
            );
    '''

    cur.execute(drop_table)
    cur.execute(missing_person_table)

    conn.commit()
    conn.close()

In [9]:
def update_mp_table(csv_file):
    '''
    Add information from movie_info.csv to populate the movieInfo table in the movie database. 
    Connect with director table to pull in foreign key information for directorId column.

    Parameters
    ----------
    None

    Returns
    -------
    None
    '''

    conn = sqlite3.connect('fbi.db')
    cur = conn.cursor()

    insert_data = '''
        INSERT INTO missingPerson
        VALUES (NULL,?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    '''

    # Open movies csv file and skip header
    with open(csv_file, "r") as csvfile:
        csvreader = csv.reader(csvfile)
        #skip header row
        next(csvreader)

        #Connect id from director table to movieInfo table on director name/id 
        for row in csvreader:
            #Insert data from csv into database table
            cur.execute(insert_data, (
                row[0], #subject
                row[1], #name
                row[2], #dob
                row[3], #pob
                row[4], #age
                row[5], #sex
                row[6], #height
                row[7], #race
                row[8], #hair
                row[9], #eyeColor
                row[10], #scarsandmarks
                row[11], #publication
                row[12], #details
                row[13], #description
                row[14], #fieldOffices
                row[15], #url
                row[16], #image
                row[17], #page
            ))

    conn.commit()
    conn.close()

In [10]:
write_csv('mp_db.csv', d_list)
create_db()
update_mp_table('mp_db.csv')